In [1]:
!pip install torchmetrics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.5 MB/s eta 0:00:00


In [2]:
import os
import csv
import torch
import pickle
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm.notebook import tqdm
from google.colab import drive
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from torchmetrics import AUROC, F1Score

In [3]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
# Load training, validation and test data from CSV files
train_x = pd.read_csv('/content/train_x.csv')
train_y = pd.read_csv('/content/train_y.csv')
val_x = pd.read_csv('/content/val_x.csv')
val_y = pd.read_csv('/content/val_y.csv')
test_x = pd.read_csv('/content/test_x.csv')

In [6]:
train_data = pd.concat([train_x, train_y], axis=1)
val_data = pd.concat([val_x, val_y], axis=1)

In [7]:
train_data.fillna('', inplace=True)
val_data.fillna('', inplace=True)
test_x.fillna('', inplace=True)

In [8]:
# Import required libraries for the DistilBERT model and evaluation metrics

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.metrics import accuracy_score


In [9]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [15]:
from datasets import Dataset as HFDataset

# Convert pandas DataFrames to Hugging Face Dataset format for better compatibility
# Assumes train_data and val_data contain 'string' (text) and 'y' (label) columns

hf_train = HFDataset.from_pandas(train_data)
hf_val = HFDataset.from_pandas(val_data)

# Initialize the DistilBERT tokenizer with the base uncased model

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Define tokenization function that:
# Converts text to tokens
#  Truncates sequences to max length of 256


def tokenize_function(batch):
    return tokenizer(
        batch["string"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

# Apply the tokenizer in a streaming/batched way
hf_train = hf_train.map(tokenize_function, batched=True)
hf_val = hf_val.map(tokenize_function, batched=True)

# Rename column "y" to "labels" for the Trainer
hf_train = hf_train.rename_column("y", "labels")
hf_val = hf_val.rename_column("y", "labels")

# Set PyTorch format
hf_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
hf_val.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/269038 [00:00<?, ? examples/s]

Map:   0%|          | 0/45180 [00:00<?, ? examples/s]

In [16]:
from transformers import Trainer, TrainingArguments

from transformers import DistilBertConfig

config = DistilBertConfig(dropout=0.3, attention_dropout=0.3)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", config=config)

# - Training hyperparameters (epochs, batch sizes, learning rate)
# - Optimization settings (weight decay, warmup steps)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.03,
    warmup_steps=500,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    fp16=True,
)

# Initialize the trainer with the model, training arguments, and datasets

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train,
    eval_dataset=hf_val
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer.train()  # Start the training process

Epoch,Training Loss,Validation Loss
1,0.196900,0.219557
2,0.235200,0.219428
3,0.103900,0.221451


TrainOutput(global_step=50445, training_loss=0.19790832232360808, metrics={'train_runtime': 5795.7562, 'train_samples_per_second': 139.259, 'train_steps_per_second': 8.704, 'total_flos': 5.345814599941939e+16, 'train_loss': 0.19790832232360808, 'epoch': 3.0})

In [18]:
# Make predictions
predictions = trainer.predict(hf_val)
probs = torch.softmax(torch.tensor(predictions.predictions), dim=1)
pred_probs = probs[:, 1].numpy()  # Probability of positive class

In [19]:
def worst_group_accuracy(prediction, y):
    """
    Compute the worst group accuracy across demographic groups and toxicity labels.

    Arguments:
        prediction [pandas.DataFrame]: dataframe with 2 columns (index and pred)
        y [pandas.DataFrame]: dataframe containing the metadata
    Returns:
        wga [float]: worst group accuracy
    """
    y.loc[prediction.index, 'pred'] = prediction.pred

    categories = ['male', 'female', 'LGBTQ', 'christian', 'muslim', 'other_religions', 'black', 'white']
    accuracies = []
    for category in categories:
        for label in [0, 1]:
            group = y.loc[y[category] == label]
            group_accuracy = (group['y'] == (group['pred'] > 0.5)).mean()
            accuracies.append(group_accuracy)
    wga = np.min(accuracies)
    return wga

In [20]:
# Create DataFrame with predictions
predictions_df = pd.DataFrame({
    'index': val_data.index,
    'pred': pred_probs
}).set_index('index')


In [21]:
val_df = hf_val.to_pandas()

In [22]:
# Calculate and print the worst group accuracy metric
wga = worst_group_accuracy(predictions_df, val_data.copy())
print(f"Worst Group Accuracy: {wga:.4f}")

Worst Group Accuracy: 0.7742


In [23]:
hf_test = HFDataset.from_pandas(test_x)

# Apply the same tokenization function used for training and validation data
hf_test = hf_test.map(tokenize_function, batched=True)

# Set the format to PyTorch
hf_test.set_format("torch", columns=["input_ids", "attention_mask"])

# Now use hf_test for prediction
test_predictions = trainer.predict(hf_test)

Map:   0%|          | 0/133782 [00:00<?, ? examples/s]

In [24]:
# Convert predictions to a torch tensor for softmax
logits = torch.tensor(test_predictions.predictions)

# Apply softmax
probs = torch.softmax(logits, dim=1).numpy()  # shape: [num_samples, 2]

# Probability of the positive class (class index = 1)
pred_probs = probs[:, 1]

# Binary predictions using 0.5 threshold
pred_labels = (pred_probs > 0.5).astype(int)

In [25]:
# 1. Add your predictions to the test_x dataframe (renamed from test_data)
test_x["pred"] = pred_labels
test_x["ID"] = test_x["index"]
test_x[["ID", "pred"]].to_csv("submission_distilBert.csv", index=False)